<a href="https://colab.research.google.com/github/tinimini12/optiontrade/blob/colab/covered_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import pytz
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

def get_option_chain(ticker, expiry):
    stock = yf.Ticker(ticker)
    return stock.option_chain(expiry)

def get_stock_data(ticker, period="3mo"):
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period)
    return hist

def calculate_weekly_fluctuation(stock_data):
    stock_data['Week'] = stock_data.index.to_period('W')
    weekly_data = stock_data.resample('W-FRI').last()
    weekly_data['Weekly Change %'] = weekly_data['Close'].pct_change() * 100
    return weekly_data

def optimal_strike(expiry_options, expected_stock_price, next_earnings_date, next_ex_dividend_date, expiration_date):
    calls = expiry_options.calls
    if 'impliedVolatility' in calls.columns:
        # Filter out in-the-money options
        filtered_calls = calls[calls['strike'] > expected_stock_price]

        # Sort the filtered options by implied volatility in ascending order
        sorted_calls = filtered_calls.sort_values(by='impliedVolatility')

        # Remove the options that are close to earnings date or ex-dividend date
        one_week_before_expiration = expiration_date - timedelta(weeks=1)
        if next_earnings_date and one_week_before_expiration <= next_earnings_date:
            sorted_calls = sorted_calls[sorted_calls.index != next_earnings_date.strftime('%Y-%m-%d')]
        if next_ex_dividend_date and one_week_before_expiration <= next_ex_dividend_date:
            sorted_calls = sorted_calls[sorted_calls.index != next_ex_dividend_date.strftime('%Y-%m-%d')]

        # Select the first option from the filtered and sorted list as the optimal strike price
        if not sorted_calls.empty:
            optimal = sorted_calls.iloc[0]
            return optimal['strike'], optimal['lastPrice'], optimal['impliedVolatility']
        else:
            print("No optimal strike price found.")
            return None, None, None
    else:
        print("Implied Volatility column not available.")
        return None, None, None


def get_next_earnings_date(ticker):
    stock = yf.Ticker(ticker)
    earnings_dates = stock.earnings_dates.index
    current_time = pd.Timestamp.now(tz=earnings_dates.tz)
    future_earnings_dates = earnings_dates[earnings_dates > current_time]
    return future_earnings_dates.min() if not future_earnings_dates.empty else None

def get_next_ex_dividend_date(ticker):
    stock = yf.Ticker(ticker)
    dividends = stock.dividends.index
    current_time = pd.Timestamp.now(tz=dividends.tz)
    future_dividends = dividends[dividends > current_time]
    return future_dividends.min() if not future_dividends.empty else None

def main():
    ticker = input("Enter the ticker symbol: ")
    num_contracts = int(input("Enter the number of contracts: "))
    weeks_from_now = int(input("Enter the number of weeks from now for the expiration date: "))
    stock = yf.Ticker(ticker)
    current_price = stock.history(period='1d')['Close'].iloc[-1]

    expected_price_change_percent = float(input("Enter the expected stock price change (%) from current price: "))
    expected_price = current_price * (1 + expected_price_change_percent / 100)

    next_earnings_date = get_next_earnings_date(ticker)
    next_ex_dividend_date = get_next_ex_dividend_date(ticker)

    if next_earnings_date:
        print(f"Next earnings date for {ticker}: {next_earnings_date.strftime('%Y-%m-%d')}")
    else:
        print(f"No upcoming earnings date found for {ticker}.")

    if next_ex_dividend_date:
        print(f"Next ex-dividend date for {ticker}: {next_ex_dividend_date.strftime('%Y-%m-%d')}")
    else:
        print(f"No upcoming ex-dividend date found for {ticker}.")

    today = datetime.today()
    expiration_date = (today + timedelta(weeks=weeks_from_now)).date()

    if next_earnings_date:
        next_earnings_date = next_earnings_date.to_pydatetime().date()

    while next_earnings_date and abs((expiration_date - next_earnings_date).days) < 7:
        expiration_date += timedelta(weeks=1)

    while next_ex_dividend_date and abs((expiration_date - next_ex_dividend_date).days) < 7:
        expiration_date += timedelta(weeks=1)

    expiration_date = expiration_date - timedelta(days=expiration_date.weekday() - 4)  # Adjust to nearest Friday

    option_chain = get_option_chain(ticker, expiration_date.strftime('%Y-%m-%d'))
   #print("Available columns in options data:", option_chain.calls.columns)
    #strike_price, premium, implied_volatility = optimal_strike(option_chain, expected_price)
    strike_price, premium, implied_volatility = optimal_strike(option_chain, expected_price, next_earnings_date, next_ex_dividend_date, expiration_date)

    if strike_price is None:
        print("No optimal strike price found.")
        return

    stock_data = get_stock_data(ticker)
    weekly_fluctuation = calculate_weekly_fluctuation(stock_data)



    total_premium = premium * 100 * num_contracts
    total_stock_value = current_price * 100 * num_contracts
    print(f"Expirataion date: {expiration_date}")
    print(f"current stock price: {current_price}")
    print(f"Expected stock price: {expected_price}")
    print(f"\nOptimal strike price: {strike_price}")
    print(f"Premium per contract: ${premium}")
    print(f"Total premium to be collected: ${total_premium}")
    print(f"Total value of stock held: ${total_stock_value}")
    if implied_volatility is not None:
        print(f"Implied Volatility for optimal strike price: {implied_volatility}")
    # plt.figure(figsize=(10, 5))
    # plt.plot(weekly_fluctuation.index, weekly_fluctuation['Weekly Change %'], marker='o')
    # plt.title('Weekly Stock Price Fluctuation (%)')
    # plt.xlabel('Date')
    # plt.ylabel('Weekly Change (%)')
    # plt.grid(True)
    # plt.show()

if __name__ == "__main__":
    main()


Enter the ticker symbol: tsla
Enter the number of contracts: 10
Enter the number of weeks from now for the expiration date: 2
Enter the expected stock price change (%) from current price: 10
Next earnings date for tsla: 2024-06-13
No upcoming ex-dividend date found for tsla.
Expirataion date: 2024-06-28
current stock price: 178.0800018310547
Expected stock price: 195.88800201416018

Optimal strike price: 200.0
Premium per contract: $2.52
Total premium to be collected: $2520.0
Total value of stock held: $178080.0018310547
Implied Volatility for optimal strike price: 0.4880422290039063


<ipython-input-1-d8e774d188a8>:17: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['Week'] = stock_data.index.to_period('W')


In [ ]:
pip install yfinance plotly


In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime, timedelta

def plot_stock_chart(ticker):
    # Calculate the date three months ago from today
    end_date = datetime.now()
    start_date = end_date - timedelta(days=90)

    # Fetch the stock data
    stock_data = yf.download(ticker, start=start_date, end=end_date)

    # Check if data is fetched successfully
    if stock_data.empty:
        print(f"No data found for ticker symbol {ticker}")
        return

    # Create the interactive plot
    fig = go.Figure()

    # Add the stock price line
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'],
                             mode='lines+markers',
                             name=ticker,
                             text=[f"Date: {date}<br>Close: {close}" for date, close in zip(stock_data.index, stock_data['Close'])],
                             hoverinfo='text'))

    # Update layout for better visuals
    fig.update_layout(
        title=f"{ticker} Stock Price - Last 3 Months",
        xaxis_title='Date',
        yaxis_title='Close Price (USD)',
        hovermode='x unified',
        template='plotly_dark'
    )

    # Show the plot
    fig.show()

# Input the ticker symbol
ticker_symbol = input("Enter the ticker symbol: ").upper()
plot_stock_chart(ticker_symbol)


Enter the ticker symbol: tsla


[*********************100%%**********************]  1 of 1 completed
